# Combine the weatherstation and fix problems

In [2]:
import numpy as np 
import pandas as pd
from os import listdir
from os.path import isfile, join, isdir
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set_theme()
import matplotlib.pyplot as plt
import pickle
import re
import math
from datetime import datetime, timedelta
from tests import *
import sys
from tools import *
import numpy_indexed as npi
import pickle
import pandas as pd
from tools import *
from matplotlib.pyplot import figure

In [72]:
files = sorted(listdir("/home/datasets4/stein/jena_experiment/OK_Download"))

In [ ]:
/home/datasets4/stein/jena_experiment/Weatherstation

In [10]:
variables = pd.read_excel("/home/datasets4/stein/jena_experiment/Weatherstation/Weatherstation_variables_2.xlsx", engine="openpyxl")

In [11]:
variables["Variable"].values

array(['DateTime', 'p', 'T', 'T1', 'T2', 'rh', 'rh1', 'rh2', 'sh', 'sh1',
       'sh2', 'Tpot', 'Tdew', 'VPmax', 'VPact', 'VPdef', 'H2OC', 'rho',
       'wv', 'wd', 'rain1', 'rain2', 'Srain1', 'Srain2', 'Tpyr', 'TDR',
       'TUR', 'SWDR', 'SWUR', 'Albedo', 'SDUR', 'SDIR', 'LWDR', 'LWUR',
       'TRAD', 'PAR', 'Rn', 'SHF1', 'SHF2', 'SHF3', 'SHF4', 'SHF5',
       'SHFM', 'ST002', 'ST004', 'ST008', 'ST016', 'ST032', 'ST064',
       'ST128', 'SM008', 'SM016', 'SM032', 'SM064', 'SM128', 'Tlog',
       'Heating1', 'Heating2', 'Cal', 'dt', 'dsh'], dtype=object)

In [75]:
a = pd.read_excel("/home/datasets4/stein/jena_experiment/OK_Download/" + files[0])
for x in files[1:-1]: 
    a = pd.concat([a,pd.read_excel("/home/datasets4/stein/jena_experiment/OK_Download/" + x)])

In [76]:
# b has all columns doubled with some weird 30minute resolution data. Just delete doubles. Seems to be a processing mistake (no Time gets lost)
b = pd.read_excel("/home/datasets4/stein/jena_experiment/OK_Download/" + files[-1])
for x in list(b.columns): 
    if ".1" in x:
        b.drop([x], axis=1, inplace=True)
a = pd.concat([a,b])
a.reset_index(inplace=True)

In [77]:
#remaining columns
for x in list(a.columns):
    if x not in variables["Fullname"].values: 
        print(x)

index
Tpot1 (K)
Tdew1 (degC)
VPmax1 (mbar)
VPact1 (mbar)
VPdef1 (mbar)
H2OC1 (mmol/mol)
rho1 (g/m**3)
Fan (Hz)
Batt (V)
Pdown (%)
Unnamed: 53


In [78]:
#lets check if the remaining duplicates can be joined

In [79]:
duplicates = ["Tpot1 (K)", "Tdew1 (degC)" ,"VPmax1 (mbar)", "VPact1 (mbar)", "VPdef1 (mbar)", "H2OC1 (mmol/mol)", "rho1 (g/m**3)"]
real = ["Tpot (K)", "Tdew (degC)" ,"VPmax (mbar)", "VPact (mbar)", "VPdef (mbar)", "H2OC (mmol/mol)", "rho (g/m**3)"]

In [80]:
for x in range(len(real)):
    print(sum((~np.isnan(a[duplicates[x]])) & (~np.isnan(a[real[x]]))))
# No double entries. Lets join them

0
0
0
0
0
0
0


In [81]:
for x in range(len(real)):
    a.loc[~np.isnan(a[duplicates[x]]),[real[x]]] = a.loc[~np.isnan(a[duplicates[x]]),[duplicates[x]]].values
    a.drop([duplicates[x]], axis=1, inplace=True)
    print(sum(np.isnan(a[real[x]])))
# One line still NaN

1
1
1
1
1
1
1


In [82]:
# Drop trash
a.drop(["index"], axis=1, inplace=True)
a.drop(["Unnamed: 53"], axis=1, inplace=True)

In [83]:
#remaining columns
for x in list(a.columns):
    if x not in variables["Fullname"].values: 
        print(x)

Fan (Hz)
Batt (V)
Pdown (%)


In [84]:
np.sum(np.isnan(a[["Fan (Hz)","Batt (V)","Pdown (%)"]].values),axis=0)
#started lated but consistent

array([925984, 925984, 925984])

In [85]:
for x in variables["Fullname"].values:
    if x not in a.columns:
        print(x)
#sum of rain is missing but can be added later. Everything else is good. 

Srain1 (mm)
Srain2 (mm)


In [86]:
# Drop seconds from time
a.loc[:,"Date Time"] = a.loc[:,"Date Time"].astype('datetime64[m]')

In [87]:
a 

,Date Time,p (mbar),T1 (degC),T2 (degC),rh1 (%),rh2 (%),sh1 (g/kg),sh2 (g/kg),wv (m/s),wd (deg),...,VPact (mbar),VPdef (mbar),H2OC (mmol/mol),rho (g/m**3),Cal (cnt),dt (K),dsh (g/kg),Fan (Hz),Batt (V),Pdown (%)
0,2002-05-24 13:40:00,994.70,18.61,-9999.0,63.17,-9999.0,8.53,-9999.0,-9999.00,-9999.0,...,13.57,7.91,13.64,1181.54,NaN,NaN,NaN,NaN,NaN,NaN
1,2002-05-24 13:50:00,994.83,18.59,-9999.0,62.33,-9999.0,8.40,-9999.0,-9999.00,-9999.0,...,13.37,8.08,13.44,1181.87,NaN,NaN,NaN,NaN,NaN,NaN
2,2002-05-24 14:00:00,994.74,19.39,-9999.0,61.60,-9999.0,8.73,-9999.0,-9999.00,-9999.0,...,13.89,8.66,13.96,1178.29,NaN,NaN,NaN,NaN,NaN,NaN
3,2002-05-24 14:10:00,994.73,19.45,-9999.0,58.37,-9999.0,8.30,-9999.0,-9999.00,-9999.0,...,13.21,9.42,13.28,1178.34,NaN,NaN,NaN,NaN,NaN,NaN
4,2002-05-24 14:20:00,994.48,19.38,-9999.0,60.99,-9999.0,8.64,-9999.0,-9999.00,-9999.0,...,13.74,8.79,13.82,1178.09,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997641,2021-05-12 15:10:00,993.45,NaN,NaN,NaN,NaN,NaN,NaN,1.80,297.3,...,13.79,0.53,13.89,1206.03,NaN,NaN,NaN,162.8,12.78,0.0
997642,2021-05-12 15:20:00,993.40,NaN,NaN,NaN,NaN,NaN,NaN,1.77,286.3,...,13.78,0.54,13.87,1205.97,NaN,NaN,NaN,162.8,12.78,0.0
997643,2021-05-12 15:30:00,993.38,NaN,NaN,NaN,NaN,NaN,NaN,2.02,287.2,...,13.73,0.60,13.82,1205.93,NaN,NaN,NaN,162.8,12.78,0.0
997644,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
gap = a["Date Time"][1] - a["Date Time"][0]
for x in range(len(a["Date Time"])-1):
    if (a["Date Time"][x+1] - a["Date Time"][x]) == gap:
        pass
    else:
        print(x)
        print(a["Date Time"][x+1] - a["Date Time"][x])

944179
0 days 00:09:00
944189
0 days 00:11:00
997643
NaT
997644
NaT


In [89]:
a.loc[944179:944190,["Date Time"]]  # thats fine

,Date Time
944179,2020-05-06 08:50:00
944180,2020-05-06 08:59:00
944181,2020-05-06 09:09:00
944182,2020-05-06 09:19:00
944183,2020-05-06 09:29:00
944184,2020-05-06 09:39:00
944185,2020-05-06 09:49:00
944186,2020-05-06 09:59:00
944187,2020-05-06 10:09:00
944188,2020-05-06 10:19:00


In [90]:
a.loc[997643:997645,["Date Time"]] #remove this line

,Date Time
997643,2021-05-12 15:30:00
997644,NaT
997645,2021-05-12 15:40:00


In [91]:
a.drop(997644,inplace = True)
a.reset_index(inplace=True, drop=True)

In [92]:
a.to_csv("/home/datasets4/stein/station_processed/raw_station.csv", index=False)